In [ ]:
!wget https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py

--2021-08-14 10:50:35--  https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1128 (1.1K) [text/plain]
Saving to: ‘CoNLL2Spacy.py’

CoNLL2Spacy.py      100%[===================>]   1.10K  --.-KB/s    in 0s      

2021-08-14 10:50:35 (73.9 MB/s) - ‘CoNLL2Spacy.py’ saved [1128/1128]



In [ ]:
from CoNLL2Spacy import *

In [ ]:
file = open("nervalweather.txt", "r",encoding = "utf-8") 
valList = []
for line in file:
    valList.append(line[:-1])
valList[:15]

['आज U-date',
 'मौसम O',
 'कैसा O',
 'है O',
 '',
 'क्या O',
 'हरिद्वार U-location',
 'में O',
 'बहुत O',
 'ठंड U-weather_type',
 'है O',
 'अभी U-date',
 '',
 'आज U-date',
 'इतनी O']

In [ ]:
TEST_DATA = conll2spacy(valList)

In [ ]:
file = open("nertrnweather.txt", "r",encoding = "utf-8")  
trainList = []
for line in file:
    trainList.append(line[:-1])
trainList[:15]

['अरे O',
 'सारथी O',
 'जम्मू U-location',
 'कैसा O',
 'मौसम O',
 'है O',
 '',
 'अभी U-date',
 'दिल्ली U-location',
 'में O',
 'कैसा O',
 'मौसम O',
 'है O',
 '',
 'आज U-date']

In [ ]:
TRAIN_DATA = conll2spacy(trainList)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# downloading fasttext hindi vector
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec

--2021-08-14 10:55:53--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 416644179 (397M) [binary/octet-stream]
Saving to: ‘wiki.hi.vec’

wiki.hi.vec         100%[===================>] 397.34M  22.8MB/s    in 18s     

2021-08-14 10:56:11 (22.5 MB/s) - ‘wiki.hi.vec’ saved [416644179/416644179]



In [ ]:
import spacy
import numpy
import pickle

In [ ]:
lang = "hi"
vectors_loc = "wiki.hi.vec"
nlp = spacy.blank(lang)    
with open(vectors_loc, "rb") as file_:        
    header = file_.readline()        
    nr_row, nr_dim = header.split()        
    nlp.vocab.reset_vectors(width=int(nr_dim))        
    for line in file_:            
        line = line.rstrip().decode("utf8")            
        pieces = line.rsplit(" ", int(nr_dim))            
        word = pieces[0]            
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
        nlp.vocab.set_vector(word, vector)  # add the vectors to the vocab   

In [ ]:
text = "भारी बारिश के कारण आज कार्यालय बंद रहेगा"    
doc = nlp(text)    
print("similarity btw",doc[0] , "and", doc[3]," :-", doc[0].similarity(doc[3]))

similarity btw भारी and कारण  :- 0.41513643


22

In [ ]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [ ]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):

    # loading hindi model and using vector from fasttext
    lang = "hi"
    vectors_loc = "wiki.hi.vec"
    modiner = spacy.blank(lang)    
    with open(vectors_loc, "rb") as file_:        
        header = file_.readline()        
        nr_row, nr_dim = header.split()        
        modiner.vocab.reset_vectors(width=int(nr_dim))        
        for line in file_:            
            line = line.rstrip().decode("utf8")            
            pieces = line.rsplit(" ", int(nr_dim))            
            word = pieces[0]            
            vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
            modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab    

#     modiner = spacy.blank('en')  # create blank Language class
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in modiner.pipe_names:
        ner = modiner.create_pipe('ner')
        modiner.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(2.0, 16.0, 1.01))
            for batch in batches:
                texts, annotations = zip(*batch)
                modiner.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score on test data
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
            

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:                    
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                
                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)
                
    print("-----Best Model is Saved-----")
     


In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores


In [ ]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

In [ ]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [ ]:
# traning the model with 100 iterations 

train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "hindiNER")

Starting iteration 0
{'ner': 802.1098836921155}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 1
{'ner': 516.0336170627515}
Current Score :- 69.76744186046511 Precision  :- 73.17073170731707 Recall  :- 66.66666666666666
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 2
{'ner': 340.9989783267651}
Current Score :- 79.54545454545455 Precision  :- 81.3953488372093 Recall  :- 77.77777777777779
Best Sccore :-  69.76744186046511
------------------------------------
Starting iteration 3
{'ner': 254.71461708675793}
Current Score :- 97.77777777777777 Precision  :- 97.77777777777777 Recall  :- 97.77777777777777
Best Sccore :-  79.54545454545455
------------------------------------
Starting iteration 4
{'ner': 178.68077754300543}
Current Score :- 98.9010989010989 Precision  :- 97.82608695652173 Recall  :- 100.0
Best Sccore :-  97.777777777

In [ ]:
# loading the saved model
pnlp = loadNERModel("hindiNER")

In [ ]:
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 98.9010989010989
Precision of Model is :- 97.82608695652173
Recall of Model is :- 100.0


In [ ]:
from spacy import displacy

In [ ]:
testcase = pnlp(TEST_DATA[0][0])
displacy.render(testcase, style='ent', jupyter=True)

In [ ]:

for i in range(0,len(TRAIN_DATA)):
    testcase = pnlp(TRAIN_DATA[i][0])
    displacy.render(testcase, style='ent', jupyter=True)
    

In [ ]:
for i in range(0,len(TEST_DATA)):
    try:        
        testcase = pnlp(TEST_DATA[i][0])
        displacy.render(testcase, style='ent', jupyter=True)
        
    except:
        a=None

In [ ]:
data = TRAIN_DATA + TEST_DATA

len(data)

224

In [ ]:

TRAIN_DATA = data[:50]
TEST_DATA = data[50:]

In [ ]:

# traning the model with 100 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "hindiNER1")

Starting iteration 0
{'ner': 256.2045544683933}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 1
{'ner': 178.38037136849016}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 2
{'ner': 170.2218750147149}
Current Score :- 0.5102040816326531 Precision  :- 100.0 Recall  :- 0.2557544757033248
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 3
{'ner': 159.67436236562207}
Current Score :- 34.0080971659919 Precision  :- 81.55339805825243 Recall  :- 21.483375959079286
Best Sccore :-  0.5102040816326531
------------------------------------
Starting iteration 4
{'ner': 138.03088774433127}
Current Score :- 60.71987480438186 Precision  :- 78.2258064516129 Recall  :- 49.61636828644501
Best Sccore :-  34.0080971659919
---------------------

In [ ]:
# loading the saved model
pnlp = loadNERModel("hindiNER")
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 94.5945945945946
Precision of Model is :- 91.8854415274463
Recall of Model is :- 97.46835443037975
